# Introdução

Este trabalho tem como objetivo analisar uma base de dados com informações de solo e clima, e relacioná-las com o tipo de cultura agrícola mais adequado para cada condição. A partir da análise exploratória e descritiva, será possível identificar padrões e construir modelos preditivos que auxiliem na tomada de decisão sobre o cultivo ideal.

**Nome:** Wanderlei Grizolli  
**RM:** RMxxxxxx  
**Fase:** 3  
**Capítulo:** 2

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# Ler o dataset
df = pd.read_csv('Atividade_Cap10_produtos_agricolas.csv')
# Mostrar algumas informações iniciais
display(df.shape, df.head())

## Análise Exploratória

Nesta etapa, exploramos a estrutura da base de dados, verificando o número de registros, tipos de variáveis, presença de valores nulos e distribuição das classes. A base contém 2.200 registros e 8 variáveis, sendo 7 numéricas e 1 categórica (label), que representa o tipo de cultura.

In [ ]:
# Informações gerais e estatísticas
df.info()
df.describe()

In [ ]:
# Distribuição das classes
display(df['label'].value_counts())

## Análise Descritiva

A seguir, apresentamos gráficos que ajudam a entender o comportamento das variáveis:

- **Histogramas**: mostram a distribuição de cada variável numérica.
- **Mapa de calor de correlação**: revela relações entre variáveis, como a correlação positiva entre temperatura e umidade.
- **Boxplots por cultura**: permitem visualizar como cada variável varia entre os diferentes tipos de cultura.

Essas visualizações ajudam a identificar padrões e possíveis outliers.

In [ ]:
# Histogramas das variáveis contínuas
fig, axes = plt.subplots(2, 4, figsize=(20,10))
for ax, col in zip(axes.flatten(), df.columns[:-1]):
    sns.histplot(df[col], kde=True, ax=ax)
    ax.set_title(f'Distribuição de {col}')
fig.tight_layout()
plt.show()

In [ ]:
# Mapa de calor de correlação
plt.figure(figsize=(10,8))
sns.heatmap(df.drop('label', axis=1).corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação das Variáveis Contínuas')
plt.show()

In [ ]:
# Boxplots para comparar a distribuição dos atributos entre as classes
selected_vars = ['N','P','K','temperature','humidity','ph','rainfall']
fig, axes = plt.subplots(len(selected_vars), 1, figsize=(12, 4*len(selected_vars)))
for ax, col in zip(axes, selected_vars):
    sns.boxplot(x='label', y=col, data=df, ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_title(f'Boxplot de {col} por tipo de cultura')
fig.tight_layout()
plt.show()

## Perfil Ideal de Solo e Clima

Definimos o perfil ideal como a média geral das variáveis no conjunto de dados. Esse perfil serve como referência para comparar com culturas específicas.


In [ ]:
# Cálculo do perfil ideal (médias de cada variável)
overall_means = df.drop('label', axis=1).mean()
overall_means

Comparamos três culturas: **arroz**, **milho** e **grão-de-bico**. Observamos que:

- **Arroz** prefere alta umidade e alta precipitação, com temperatura moderada.
- **Milho** tem preferência por solos com alto nitrogênio, mas menor umidade.
- **Grão-de-bico** se destaca por tolerar baixa umidade e temperaturas mais amenas, além de exigir maior quantidade de potássio.

Essas diferenças mostram como cada cultura se adapta a diferentes condições ambientais.

In [ ]:
# Comparaçao de perfis de culturas específicas
# Substitua pelos nomes das culturas de interesse
training_products = ['rice','maize','chickpea']
crop_means = {p: df[df['label'] == p].drop('label', axis=1).mean() for p in training_products}
crop_means

In [ ]:
# Plot comparativo das médias de variáveis das culturas selecionadas versus média geral
prod_means_df = pd.DataFrame(crop_means).T
prod_means_df['overall'] = overall_means
prod_means_df.plot(kind='bar', figsize=(12,8))
plt.title('Comparação entre perfis médios de produtos e o perfil ideal')
plt.ylabel('Valor médio')
plt.xticks(rotation=0)
plt.show()

## Modelos Preditivos

Construímos seis modelos de classificação para prever o tipo de cultura com base nas condições de solo e clima:

- K-Nearest Neighbors (KNN)
- Regressão Logística
- Support Vector Machine (SVM)
- Árvore de Decisão
- Random Forest
- Naive Bayes

Os modelos foram avaliados com métricas de acurácia e relatório de classificação. Os melhores resultados foram obtidos com **Random Forest** e **Naive Bayes**, ambos com acurácia superior a 99%.

A matriz de confusão do modelo Random Forest mostra que a maioria das culturas foi corretamente classificada, com poucos erros de predição.

In [ ]:
# Preparação de dados
X = df.drop('label', axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definição de modelos a serem testados
models = {
    'KNN': Pipeline([('scaler', StandardScaler()), ('clf', KNeighborsClassifier())]),
    'Logistic Regression': Pipeline([('scaler', StandardScaler()), ('clf', LogisticRegression(max_iter=1000))]),
    'SVM': Pipeline([('scaler', StandardScaler()), ('clf', SVC())]),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Naive Bayes': Pipeline([('scaler', StandardScaler()), ('clf', GaussianNB())])
}
# Treinar e avaliar os modelos
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    results[name] = {
        'accuracy': accuracy_score(y_test, preds),
        'report': classification_report(y_test, preds, zero_division=0)
    }
results

In [ ]:
# Matriz de confusão para o modelo Random Forest
best_model = models['Random Forest']
best_preds = best_model.predict(X_test)
cm = confusion_matrix(y_test, best_preds)
sns.heatmap(cm, annot=False, fmt='d', xticklabels=best_model.named_steps.get('clf', best_model).classes_, yticklabels=best_model.named_steps.get('clf', best_model).classes_)
plt.title('Matriz de Confusão para Random Forest')
plt.ylabel('Verdades')
plt.xlabel('Predições')
plt.show()

## Conclusão

A análise permitiu identificar padrões relevantes entre as variáveis ambientais e os tipos de cultura. Os modelos preditivos demonstraram alta performance, o que indica que os dados estão bem estruturados e separados.

**Pontos fortes**:
- Base balanceada com 100 registros por cultura.
- Alta acurácia dos modelos.
- Visualizações claras e informativas.

**Limitações**:
- A base não contém informações sobre produtividade ou rendimento.
- O perfil ideal foi definido apenas com base em médias, sem considerar variabilidade ou sazonalidade.

**Próximos passos**:
- Testar os modelos com dados reais de campo.
- Incluir variáveis econômicas ou de rendimento para uma análise mais completa.